In [1]:
from azure_authentication.customized_azure_login import CredentialFactory
from openai import AzureOpenAI
import os
from pydantic import BaseModel, Field, conint, confloat

In [2]:
import openai

In [3]:
from typing import List, Optional, Literal

In [4]:
api_version="2024-10-21"
api_base= "https://greendia-oai.openai.azure.com/"
deployment_name = "gpt-4o-mini"

Note: changing to API version 2024-10-21 allows parsing of the content but becomes way more expensive, moves up to 16 usd per request! 

In [5]:
class GRZ(BaseModel):
    value: float = Field(
        None,
        description="The numerical value of the Grundflächenzahl or 'null' if not available",
        example=0.7
    )

class GFZ(BaseModel):
    value: float = Field(
        None,
        description="The numerical value of the Geschossflächenzahl or 'null' if not available",
        example=0.2
    )


In [6]:
parser_tool = [openai.pydantic_function_tool(GFZ)]


In [7]:
client = AzureOpenAI(
    api_key=os.environ["KEY_GPT_4O_MINI"],  
    api_version=api_version,
    base_url=f"{api_base}/openai/deployments/{deployment_name}"
)

In [14]:
import base64
from mimetypes import guess_type

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"


In [9]:
def calculate_llm_calling_price(model_name, 
                                input_tokens, 
                                output_tokens):
    """
    Cost calculator
    based on prices from https://azure.microsoft.com/en-us/pricing/details/cognitive-services/openai-service/#pricing
    """

    if model_name == "gpt-4o-mini":
        return input_tokens / 1000 * 0.15 + output_tokens / 1000 * 0.6
    else:
        return -1.0

In [15]:
# Example usage
image_path = '../data/proc/building_plans_sample/split_pdf/904/904_0.jpg'
data_url = local_image_to_data_url(image_path)

In [11]:
if api_version == "2024-10-21":
    response = client.beta.chat.completions.parse(
        model=deployment_name,
        messages=[
            { 
                "role": "system", 
                "content": "You are a helpful environmental city planner." 
            },
            { 
                "role": "user",
                "content": "Extract the Geschossflächenzahl from the image.",
            },
            {
                "role": "user",
                "content": data_url  # Assuming this is a string URL to the image
            }
        ],
        response_format=GFZ,
        max_tokens=2000
    )

    result = response.choices[0].message.parsed 

else: 

    response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            { 
                "role": "system", 
                "content": "You are a helpful environmental city planner." 
            },
            { 
                "role": "user",
                "content": "Extract the Geschossflächenzahl from the image.",
            },
            {
                "role": "user",
                "content": data_url  # Assuming this is a string URL to the image
            }
        ],
        max_tokens=2000
    )
    
    result = response.choices[0].message.content

In [12]:
result

GFZ(value=0.2)

In [13]:
calculate_llm_calling_price(deployment_name, response.usage.prompt_tokens, response.usage.completion_tokens)

16.01745